<a href="https://colab.research.google.com/github/Pedro-HMC-Martins/Classificacao_de_textos/blob/main/Classificacao_de_textos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 11.4 MB/s eta 0:00:00


In [ ]:
import torch
import datasets

In [ ]:
dataset = datasets.load_dataset("okite97/news-data")
dataset

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Title', 'Excerpt', 'Category'],
        num_rows: 4686
    })
    test: Dataset({
        features: ['Title', 'Excerpt', 'Category'],
        num_rows: 828
    })
})

In [ ]:
dataset['train'][0]

{'Title': 'Uefa Opens Proceedings against Barcelona, Juventus and Real Madrid Over European Super League Plan',
 'Excerpt': 'Uefa has opened disciplinary proceedings against Barcelona, Juventus and Real Madrid over their involvement in the proposed European Super League.',
 'Category': 'sports'}

In [ ]:
dataset['train'] = dataset['train'].remove_columns("Title")
dataset['test'] = dataset['test'].remove_columns("Title")
dataset

DatasetDict({
    train: Dataset({
        features: ['Excerpt', 'Category'],
        num_rows: 4686
    })
    test: Dataset({
        features: ['Excerpt', 'Category'],
        num_rows: 828
    })
})

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
def map_labels(example):
    example['id_labels'] = label_encoder.transform([example['Category']])[0]
    return example

In [ ]:
train_data = dataset['train']
test_data = dataset['test']

label_encoder = LabelEncoder()
label_encoder.fit(train_data['Category'])

dataset = dataset.map(map_labels)

Map:   0%|          | 0/4686 [00:00<?, ? examples/s]

Map:   0%|          | 0/828 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Excerpt', 'Category', 'id_labels'],
        num_rows: 4686
    })
    test: Dataset({
        features: ['Excerpt', 'Category', 'id_labels'],
        num_rows: 828
    })
})

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
text_train = dataset['train']['Excerpt']
labels_train = dataset['train']['id_labels']
text_test = dataset['test']['Excerpt']
labels_test = dataset['test']['id_labels']

In [ ]:
text_train[0]

'Uefa has opened disciplinary proceedings against Barcelona, Juventus and Real Madrid over their involvement in the proposed European Super League.'

In [ ]:
vectorizer_text = CountVectorizer(lowercase=True)
X_text_vec_train = vectorizer_text.fit_transform(text_train)
X_text_vec_test = vectorizer_text.transform(text_test)

In [ ]:
X_text_vec_train[0]

<1x10635 sparse matrix of type '<class 'numpy.int64'>'
	with 20 stored elements in Compressed Sparse Row format>

In [ ]:
X_text_vec_train = torch.FloatTensor(X_text_vec_train.toarray())
X_text_vec_test = torch.FloatTensor(X_text_vec_test.toarray())
labels_train = torch.LongTensor(labels_train)
labels_test = torch.LongTensor(labels_test)

In [ ]:
print(dataset['test']['Excerpt'][0])
print(dataset['test']['Category'][0])
print(dataset['test']['id_labels'][0])

Cross River State Governor, Ben Ayade has defected from the People's Democratic Party to the governing All Progressives Congress (APC).
politics
3


In [ ]:
import numpy as np
unique_labels = np.unique(dataset['train']['Category'])

In [ ]:
unique_labels

array(['business', 'entertainment', 'health', 'politics', 'sports',
       'tech'], dtype='<U13')

In [ ]:
from torch import nn
import torch.nn.functional as F

len_TFIDF = X_text_vec_train.shape[1]
HIDDEN_DIM = 16

class resumeClassifier(nn.Module):
  def __init__(self):
    super(resumeClassifier,self).__init__()


    self.fc1 = nn.Linear(len_TFIDF, HIDDEN_DIM)
    self.fc2 = nn.Linear(HIDDEN_DIM, 6)

  def forward(self, input):

    output = self.fc1(input)
    output = F.relu(output)
    output = self.fc2(output)
    probabilities = F.softmax(output, dim=1)

    return probabilities


In [ ]:
model = resumeClassifier()

In [ ]:
model(X_text_vec_train)

tensor([[0.2095, 0.1729, 0.1425, 0.1617, 0.1603, 0.1531],
        [0.2106, 0.1731, 0.1402, 0.1636, 0.1593, 0.1532],
        [0.2091, 0.1710, 0.1424, 0.1625, 0.1605, 0.1545],
        ...,
        [0.2073, 0.1720, 0.1438, 0.1616, 0.1599, 0.1554],
        [0.2072, 0.1691, 0.1426, 0.1627, 0.1603, 0.1580],
        [0.2094, 0.1730, 0.1419, 0.1633, 0.1579, 0.1545]],
       grad_fn=<SoftmaxBackward0>)

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
num_epochs = 200

for epoch in range (num_epochs):
  model.train()
  optimizer.zero_grad()
  outputs = model(X_text_vec_train)
  loss = criterion(outputs, labels_train)
  loss.backward()
  optimizer.step()
  print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

model.eval()
with torch.no_grad():
  test_outputs = model(X_text_vec_test)
  _, predicted = torch.max(test_outputs,1)
  accuracy = torch.sum(predicted == labels_test).item() / labels_test.size(0)
  print(f'Accuracy: {accuracy * 100:.2f}%')

Epoch [1/200], Loss: 1.7887
Epoch [2/200], Loss: 1.7871
Epoch [3/200], Loss: 1.7851
Epoch [4/200], Loss: 1.7828
Epoch [5/200], Loss: 1.7803
Epoch [6/200], Loss: 1.7776
Epoch [7/200], Loss: 1.7748
Epoch [8/200], Loss: 1.7718
Epoch [9/200], Loss: 1.7688
Epoch [10/200], Loss: 1.7658
Epoch [11/200], Loss: 1.7626
Epoch [12/200], Loss: 1.7593
Epoch [13/200], Loss: 1.7559
Epoch [14/200], Loss: 1.7525
Epoch [15/200], Loss: 1.7489
Epoch [16/200], Loss: 1.7452
Epoch [17/200], Loss: 1.7414
Epoch [18/200], Loss: 1.7376
Epoch [19/200], Loss: 1.7336
Epoch [20/200], Loss: 1.7295
Epoch [21/200], Loss: 1.7253
Epoch [22/200], Loss: 1.7210
Epoch [23/200], Loss: 1.7166
Epoch [24/200], Loss: 1.7121
Epoch [25/200], Loss: 1.7075
Epoch [26/200], Loss: 1.7027
Epoch [27/200], Loss: 1.6979
Epoch [28/200], Loss: 1.6929
Epoch [29/200], Loss: 1.6879
Epoch [30/200], Loss: 1.6827
Epoch [31/200], Loss: 1.6774
Epoch [32/200], Loss: 1.6720
Epoch [33/200], Loss: 1.6665
Epoch [34/200], Loss: 1.6609
Epoch [35/200], Loss: 1